# <font size=6>Trabalho Final de Algoritmos Genéticos

___
<font size=3>Este arquivo armazena as informações sobre o trabalho semestral desenvolvido na disciplina de **redes neurais e algoritmos genéticos por alunos da Ilum Escola de Ciência.**</font>

___

<font size=3>**Alunos:** Cauê Gomes Correia dos Santos, Izaque Junior Oliveira Silva e Karla Rovedo Pascoalini.</font> 

<font size=3>**Professor:** Daniel Roberto Cassar.</font> 
___

<font size=3>O intuito deste trabalho é desenvolver um `Algoritmo Genético` que otimize a função objetivo `minimiza_energia`, que busca a menor energia de um conjunto de átomos qualquer. De acordo com o método de Huckel é possível calcular todas as energias possíveis a partir de todas as combinações possíveis dado apenas o número de átomos. A forma encontrada pelos alunos de como essa função calcula a menor energia tende a ser custosa computacionalmente, como se pode imaginar. Devido a esse problema, resolveu-se aplicar um algoritmo genético, de minimização, para que o custo computacional seja menor e que possamos obter sem testar todas as possibilidade, uma das menores energias do conjunto de átomos. Com o intuito de tornar mais claro o fato do código feito pelo aluno Cauê ser muito custoso computacionalmente, vamos demonstrar a equação que calcula a quantidade total de conformações possívels da molécula dado apenas o número de átomos:
</font>

$$\text{c_Total} = 2^{\frac{n(n-1)}{2}}$$

<font size=3>Onde cT é u número de Conformações Totais e n o número de átomos. Note que quando olhamos 
    
    Para um conjunto de 2 átomos, cT é igual a 2. 
    
    Para n = 5, cT = 1024.
    
    Para n = 7, cT = 2.09 x 10^(6) ou ~2 milhões.
    
    Para n = 10, cT = 3.51 x 10^(13) ou ~35 trilhões.
    
Ou seja, cada átomo adicionado aumenta exponencialmente a quantidade de conformações totais. E isso é muito custoso. Foi necessário a utilização do HPC para calcular mais do que 7 átomos dado o poder computacional exigido.

## 1 - Importando as bibliotecas
___ 

In [4]:
import random
import numpy as np
from scripts import create_upper_triangular_binary_matrix as create_traingular_umatrix
from scripts import testar_matrizes
from scripts import minimiza_energia
from scripts import fitness as funcao_objetivo
from scripts import initialize_population as inicializador#initialize_population
from scripts import tournament_selection

<font size=3>A função `drawn` foi desenvolvida para plotar o grafo da molécula de menor energia que resultou da função `minimiza_energia` dado um conjunto de átomos.</font>

## 2 - Código inicial não-otimizado
### "Nobody knows me like you do" - Made For Me, Muni Long.
___

<font size=3>A função `minimiza_energia` recebe a quantidade de átomos que estarão presentes na molécula. A partir desses dados ela calcula a quantidade de possíveis arranjos entre esse átomos e gera todas as possíveis matrizes, que são `binárias`. Com as matrizes geradas na função acima, a função `testa_matrizes` recebe as matrizes e calcula a energia total (`t_total`) de cada matriz e rotorna a `melhor matriz`, junto a seus `autovalores (em ordem crescente)` e a `menor energia encontrada`.</font>

In [5]:
matrizes = minimiza_energia(6)
A = testar_matrizes(matrizes)

------------------------------------------------------------ Resultado ------------------------------------------------------------
A matriz com o menor t_total associado é:
[[ 0.  0. -1. -1.  0. -1.]
 [ 0.  0.  0. -1. -1. -1.]
 [-1.  0.  0.  0. -1. -1.]
 [-1. -1.  0.  0.  0. -1.]
 [ 0. -1. -1.  0.  0. -1.]
 [-1. -1. -1. -1. -1.  0.]]
Com os autovalores [-3.4494897427831814, -0.6180339887498958, -0.6180339887498947, 1.4494897427831785, 1.6180339887498956, 1.6180339887498956]
E com o valor associado de -9.371115440565944


## 3 - Código atual otimizado
### "O acaso vai me proteger" - Epitáfio, Titãs
___

<font size=3>Na tentativa de resolver o problema de encontrar a menor energia com um custo computacional menor, desenvolvemos um código de algoritmo genético para otimizar o nosso objetivo, cuja `função objetivo` é o cálculo da energia total. Ao final, o algoritmo nos entregará o indivíduo com a menor `energia total` encontrada. É importante comentar que não necessáriamente o código devolve o mínimo global, ou seja a menor eneriga de todas as conformações, mas devolve a menor energia dadas as opções que ele seleciona.</font>

<font size=3>Nosso código contou com alterações nos operadores genéticos `cruzamento` e `mutação`, os quais adaptamos para resolver o nosso problema. Os operadores utilizados foram:</font>

<font size=3>- Seleção: Seleção por torneio com 3 indivíduos.</font>


<font size=3>- Cruzamento: Um novo cruzamento que apenas altera a matriz triangular superior.</font>


<font size=3>- Mutação: Mutação simples, porém com a restrição de somente ocorrer fora e acima da diagonal principal.</font>


<font size=3>- Hall da Fama: Indivíduo com a menor energia total encontrada.</font>


In [38]:
def genetic_algorithm(pop_size, matrix_size, generations, chance_mutacao, chance_cruzamento):
    # Initialize population
    population = inicializador(pop_size, matrix_size)
    
    for generation in range(generations):
        # Evaluate fitness of the population
        fitnesses = [funcao_objetivo(combined_matrix) for _, combined_matrix in population]
        
        new_population = []
        
        # Generate new population
        for _ in range(pop_size // 2):
            # Select parents
            parent1 = tournament_selection(population, fitnesses)
            parent2 = tournament_selection(population, fitnesses)
            
            # Crossover
            child1, child2 = crossover(parent1, parent2, chance_cruzamento)
            
            
            # Mutate
            lista_filhos = [child1[0], child2[0]]
            
            mutate_upper_triangular_binary(lista_filhos, chance_mutacao)#, child1[0] + child1[0].T)
            #child2 = (mutate_upper_triangular_binary(child2[0]), child2[0] + child2[0].T)
            
            new_population.extend([child1, child2])
            #print(new_population)
        
        # Replace old population with new population
        population = new_population
    
    # Final population fitness evaluation
    fitnesses = [funcao_objetivo(combined_matrix) for _, combined_matrix in population]
    best_individual = population[np.argmin(fitnesses)]
    
    return best_individual, fitnesses

In [48]:
pop_size = 100
matrix_size = 7
generations = 80
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.2
# Run genetic algorithm
best_solution, fit = genetic_algorithm(pop_size, matrix_size, generations, CHANCE_MUTACAO, CHANCE_CRUZAMENTO)

print("Best solution found:")
print(best_solution[1])  # Print the combined matrix
print("Fitness of the best solution:", funcao_objetivo(best_solution[1]))

Best solution found:
[[0. 1. 1. 1. 0. 0. 0.]
 [1. 0. 1. 1. 1. 1. 0.]
 [1. 1. 0. 0. 1. 0. 1.]
 [1. 1. 0. 0. 0. 1. 1.]
 [0. 1. 1. 0. 0. 1. 1.]
 [0. 1. 0. 1. 1. 0. 1.]
 [0. 0. 1. 1. 1. 1. 0.]]
Fitness of the best solution: -11.069194527751078


## Referências Bibliográficas

[1] CASSAR, D. - GA 4.2 - Notebook Descobrindo a senha.

[2] SANTOS, C. ; LIMA, F. - Hamiltonian for n-site molecules.